In [1]:
import numpy as np
import pandas as pd
import cvxpy as cp
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Load the data 
rest_df = pd.read_csv('data/test/restaurants.csv')
locs_df = pd.read_csv('data/test/locations.csv')

In [87]:
# Filter out the restaurants based on day of the week, number of deliveries per week
DAY = 'M'
def filter_day(row):
    closed_days = row['Closed'].replace(' ', '').split(',')
    if not (DAY in closed_days):
        return True
    else:
        return False
rest_df_f = rest_df[rest_df.apply(filter_day, axis=1)]

# TO DO: Filter based on number of deliveries already completed for the week using db

In [88]:
# Grab the constraints
dist_constraint = rest_df_f['Max Delivery Distance (miles)'].values
slot_constraint = rest_df_f['Max Deliveries/day'].values

In [89]:
# Generate the delivery distance matrix
distance_mat = np.random.rand(len(locs_df), len(rest_df_f))*dist_constraint

In [90]:
# Generate the revenue estimate matrix
revenue_mat = np.random.rand(len(locs_df), len(rest_df_f))*50.0

In [91]:
# Scale the revenue estimate matrix by: dilution factor, preference factor
dilution_mat = np.random.rand(len(locs_df), len(rest_df_f)) # Between 0 and 1 based on # of deliveries to location
preference_mat = np.random.rand(len(locs_df), len(rest_df_f))+1 # Between 1 and 2 
revenue_mat_s = revenue_mat * dilution_mat * preference_mat # Element wise operation

In [92]:
# Create the decision matrix
selections = cp.Variable((revenue_mat_s.shape[1], revenue_mat_s.shape[0]), boolean=True)

In [93]:
# Create the distance constraints
dis_constraint = [distance_mat[i, :]@selections[:, i] <= dist_constraint for i in range(len(locs_df))] # Distnace
# Create the column constraint
col_constraint = [cp.sum(selections, axis=1) <= slot_constraint] # Number of deliveries per day
# Create the row constraint
row_constraint = [cp.sum(selections, axis=0) <= 1] # Each slot has at most 1 scheduled restaurant
# Create the total constraint
tot_constraint = [cp.sum(selections) == min(revenue_mat_s.shape)]

In [94]:
# Concatenate together
constraints =  dis_constraint + col_constraint + row_constraint + tot_constraint

In [95]:
# Create the objective
objective = cp.Maximize(
    cp.sum(
        [revenue_mat_s[i, :]@selections[:, i] for i in range(len(locs_df))]
    )
)

In [96]:
# Solve it
knapsack_problem = cp.Problem(objective, constraints)
result = knapsack_problem.solve()

In [97]:
# Recover the schedule 
schedule = []
rest_names = rest_df_f['Name'].values
locs_names = locs_df['Name'].values
for idx, x in np.ndenumerate(selections.value):
    if x >= 1.0:
        schedule.append((locs_names[idx[1]], rest_names[idx[0]]))

In [98]:
# Display the results
print("Day of Week: %s, Estimated Revenue: $%0.2f" % (DAY, result))
print("------------------------------------------")
for s in schedule:
    print("Loc: %10s, Rest: %s" % s)

Day of Week: M, Estimated Revenue: $117.67
------------------------------------------
Loc:       CHOP, Rest: El Merkury
Loc:  Jefferson, Rest: Sidecar
Loc:        HUP, Rest: Destination Dogs
Loc:     Presby, Rest: Sabrina's Cafe
